In [4]:
import os
from fuse import FuseOSError
import errno
import requests
from bs4 import BeautifulSoup
import re
from datetime import datetime

In [16]:
def fetch_entries_from_web(parentPath):
    output = [
        {
            "name": "..",
            "size": "-",
            "is_directory": True,
            "last_updated": 0
        },
        {
            "name": ".",
            "size": "-",
            "is_directory": True,
            "last_updated": 0
        },
    ]
    # output[0]
    # {
    #   name: string,
    #   size: int(bytes)
    #   is_directory: boolean
    #   last_updated: int(datetime(yyyy, m, d, hh, mm).timestamp)
    # }
    response = requests.get("https://lysakermoen.com"+parentPath)
    soup = BeautifulSoup(response.text, 'html.parser')
    links = soup.find_all('a')
    for link in links:
        fileObject = {}
        clean_sibling = link.next_sibling.strip()
        clean_sibling = re.sub(' +', ' ', clean_sibling)
        if clean_sibling == '':
            
            output.append(fileObject)
            continue
        datestring, fileSize = re.match(r'^(\d\d-[A-Z][a-z]{2}-\d\d\d\d \d\d:\d\d) ([0-9-]+)$', clean_sibling).groups()
        fileObject["name"] = re.match(r'([^\/]+)', link.get('href')).groups()[0]
        fileObject["size"] = fileSize
        fileObject["is_directory"] = bool(re.search(r'/$', link.get('href')))
        fileObject["last_updated"] = int(datetime.strptime(datestring, "%d-%b-%Y %H:%M").timestamp())
        output.append(fileObject)
    return output


In [7]:
def find_matching_entry(entries, entryname):
    for entry in entries:
        if entry["name"] == entryname:
            return entry
    
    return None
    

In [8]:
def build_result(matching_entry):
    if matching_entry["is_directory"]:
        return dict(st_mode=(0o040000 | 0o555), st_nlink=2, st_mtime=matching_entry["last_updated"])
    else:
        return dict(st_mode=(0o100000 | 0o444), st_nlink=1, st_size=matching_entry["size"], st_mtime=matching_entry["last_updated"])

In [14]:
def getattr(path):
    result = handle_special_cases(path)
    if result:
        return result

    parentPath = os.path.dirname(path)
    entries = fetch_entries_from_web(parentPath)
    print(entries)
    entryname = os.path.basename(path)
    matching_entry = find_matching_entry(entries, entryname)
    
    if matching_entry is None:
        raise FuseOSError(errno.ENOENT)

    return build_result(matching_entry)

In [18]:
getattr("/Software/Linux/Scripts/CustomScripts/..")

[{'name': '..', 'size': '-', 'is_directory': True, 'last_updated': 0}, {'name': '.', 'size': '-', 'is_directory': True, 'last_updated': 0}, {}, {'name': 'Sanderflix.sh', 'size': '274', 'is_directory': False, 'last_updated': 1649424060}, {'name': 'installer.sh', 'size': '1379', 'is_directory': False, 'last_updated': 1649424060}, {'name': 'packettracer.sh', 'size': '352', 'is_directory': False, 'last_updated': 1649424060}]


{'st_mode': 16749, 'st_nlink': 2, 'st_mtime': 0}